# **1. Import Library**

In [13]:
from keras.datasets import cifar10, mnist
from keras.models import Model
from keras.models import Sequential
from keras.layers import Input
from keras.layers import Reshape
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import Dropout
from keras.layers import LeakyReLU
from tensorflow.keras.optimizers import Adam
import numpy as np
!mkdir generated_images

mkdir: cannot create directory ‘generated_images’: File exists


# **2. Mendefinisikan beberapa variabel**
Mendefinisikan parameter untuk membangun sebuah model deep learning yang kemungkinan digunakan dalam arsitektur seperti Generative Adversarial Network (GAN) atau Convolutional Neural Network (CNN). Gambar yang akan diproses memiliki ukuran 32x32 piksel dengan 3 saluran warna (RGB), dan bentuk gambar tersebut ditetapkan dalam variabel img_shape. Selain itu, vektor laten yang digunakan untuk mempelajari representasi tersembunyi gambar memiliki dimensi 100. Optimizer yang dipilih untuk melatih model adalah Adam, dengan learning rate sebesar 0.0002.

In [14]:
img_width = 32
img_height = 32
channels= 3
img_shape = (img_width, img_height, channels)
latent_dim = 100
adam = Adam(learning_rate=0.0002)

# **3. Membentuk generator**
Membangun generator untuk sebuah model GAN, yang berfungsi menghasilkan gambar dari vektor laten. Dimulai dengan dense layer yang mengubah input menjadi tensor, lalu menggunakan beberapa lapisan Conv2DTranspose untuk memperbesar gambar secara bertahap. Setiap lapisan dilengkapi dengan fungsi aktivasi LeakyReLU untuk menambah non-linearitas. Pada akhirnya, layer Conv2D terakhir membentuk gambar output dengan 3 channel (RGB) menggunakan aktivasi 'tanh'.

In [15]:
def build_generator():
  model = Sequential()

  # create first Layer, to receive the input
  model.add(Dense (256 * 4 * 4, input_dim = latent_dim))
  # 256 * 8 * 8; for upscaling the Layers
  #initial shape to construct into final shape

  # Create default activation function
  model.add(LeakyReLU(alpha = 0.2))

  # Create reshape Layer
  model.add(Reshape((4, 4,256)))
  # 6,8,256; reffers to first layer

  # Adding more Layers for neurons and better result
  model.add(Conv2DTranspose(128, (4,4), strides = (2,2), padding = 'same'))
  model.add(LeakyReLU(alpha= 0.2))
  model.add(Conv2DTranspose(128, (4,4), strides = (2,2), padding = 'same'))
  model.add(LeakyReLU (alpha= 0.2))
  model.add(Conv2DTranspose(128, (4,4), strides = (2,2), padding = 'same'))
  model.add(LeakyReLU(alpha= 0.2))
  # (4,4) >> filter size
  # strides (2,2) >> Convolutional Layers, that how NN understand images

  # Create Final output Layer and forming image shape
  # the shape (3, (3,3)) reffers to image shape :
  # >>> img_shape (1mg width, img_height, channels)
  model.add(Conv2D (3, (3,3), activation = 'tanh', padding = 'same'))

  model.summary()
  return model

generator = build_generator()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                      │ (None, 4096)                │         413,696 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_12 (LeakyReLU)           │ (None, 4096)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ reshape_1 (Reshape)                  │ (None, 4, 4, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose_3 (Conv2DTranspose) │ (None, 8, 8, 128)           │         524,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_13 (LeakyReLU)           │ (None, 8, 8, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose_4 (Conv2DTranspose) │ (None, 16, 16, 128)         │         262,272 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_14 (LeakyReLU)           │ (None, 16, 16, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose_5 (Conv2DTranspose) │ (None, 32, 32, 128)         │         262,272 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_15 (LeakyReLU)           │ (None, 32, 32, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_9 (Conv2D)                    │ (None, 32, 32, 3)           │           3,459 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,466,115 (5.59 MB)

 Trainable params: 1,466,115 (5.59 MB)

 Non-trainable params: 0 (0.00 B)

# **4. Mendefinisikan discriminator**
Membangun discriminator untuk GAN, yang berfungsi membedakan gambar asli dari yang palsu. Discriminator menggunakan beberapa lapisan Conv2D dengan fungsi aktivasi LeakyReLU untuk mengekstrak fitur gambar, kemudian lapisan Flatten dan Dropout untuk mencegah overfitting. Akhirnya, output dihasilkan oleh lapisan Dense dengan aktivasi sigmoid yang memberikan probabilitas apakah gambar itu nyata atau hasil buatan. Model ini dikompilasi menggunakan binary crossentropy sebagai loss dan Adam sebagai optimizer.

In [16]:
def build_discriminator():
  model = Sequential()

  # Create Input layer and filter and stride layer. That makes NN understand (mage
  model.add(Conv2D(64, (3,3), padding = 'same', input_shape = img_shape))

  # Adding activation function
  model.add(LeakyReLU(alpha = 0.2))
  model.add(Conv2D(128, (3,3), padding = 'same'))
  model.add(LeakyReLU(alpha = 0.2))
  model.add(Conv2D(128, (3,3), padding = 'same'))
  model.add(LeakyReLU(alpha = 0.2))
  model.add(Conv2D (256, (3,3), padding = 'same'))
  model.add(LeakyReLU(alpha = 0.2))
  model.add(Flatten())

  model.add(Dropout(0.4))

  # Create output Layer
  model.add(Dense(1, activation = 'sigmoid'))

  model.summary()
  return model

discriminator = build_discriminator()
discriminator.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_10 (Conv2D)                   │ (None, 32, 32, 64)          │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_16 (LeakyReLU)           │ (None, 32, 32, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_11 (Conv2D)                   │ (None, 32, 32, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_17 (LeakyReLU)           │ (None, 32, 32, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_12 (Conv2D)                   │ (None, 32, 32, 128)         │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_18 (LeakyReLU)           │ (None, 32, 32, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_13 (Conv2D)                   │ (None, 32, 32, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_19 (LeakyReLU)           │ (None, 32, 32, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 262144)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 262144)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 1)                   │         262,145 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 780,545 (2.98 MB)

 Trainable params: 780,545 (2.98 MB)

 Non-trainable params: 0 (0.00 B)

# **5. Menghubungkan Discriminator dan Generator untuk membentuk GAN**
Membangun GAN dengan menggabungkan generator dan discriminator. Discriminator dibuat tidak dapat dilatih agar hanya generator yang dilatih. Vektor laten dimasukkan ke generator untuk menghasilkan gambar, lalu gambar ini dinilai oleh discriminator. Model GAN ini dikompilasi menggunakan binary crossentropy sebagai loss dan Adam sebagai optimizer, sehingga dapat melatih generator untuk menghasilkan gambar yang realistis.

In [17]:
def build_gan(generator, discriminator):
    discriminator.trainable = False
    gan_input = Input(shape=(latent_dim,))
    generated_image = generator(gan_input)
    gan_output = discriminator(generated_image)
    gan = Model(gan_input, gan_output)

    gan.compile(loss='binary_crossentropy', optimizer=adam)
    return gan

gan = build_gan(generator, discriminator)
gan.summary()

Model: "functional_54"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_6 (InputLayer)           │ (None, 100)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ sequential_3 (Sequential)            │ (None, 32, 32, 3)           │       1,466,115 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ sequential_4 (Sequential)            │ (None, 1)                   │         780,545 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,246,660 (8.57 MB)

 Trainable params: 1,466,115 (5.59 MB)

 Non-trainable params: 780,545 (2.98 MB)